In [1]:
#import modules
import gmaps
import gmaps.geojson_geometries
import json
import pandas as pd
from api_key import g_key

In [2]:
#set up gmaps, pull in countries geometry
gmaps.configure(api_key=g_key)
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

In [3]:
#check access & dict structure in countries geometry
countries_geojson['features'][0]['properties']

{'ISO_A3': 'AFG', 'name': 'Afghanistan'}

In [4]:
#open meat consumption data
meat_df=pd.read_csv("Resources/Meat consumption.csv")

In [6]:
#check duplicates
meat_df["Country"].value_counts()

Guinea              2
Afghanistan         1
Papua New Guinea    1
Niger               1
Nigeria             1
                   ..
Grenada             1
Guadeloupe          1
Guam                1
Guatemala           1
Zimbabwe            1
Name: Country, Length: 186, dtype: int64

In [7]:
#drop duplicates
meat_clean_df = meat_df.drop_duplicates(subset="Country")

In [8]:
#pull list of countries from gmaps_geojson geometry
gmaps_countries=[]
y=0

for x in countries_geojson["features"]:
    gmaps_countries.append(countries_geojson['features'][y]['properties']['name'])
    y+=1

len(gmaps_countries)

217

In [9]:
#make both country lists into series
meat_countries = meat_clean_df["Country"]   
gmaps_country_series = pd.Series(gmaps_countries)

In [10]:
#print out country names from meat_countries that aren't in gmaps_countries
meat_tofind=meat_countries.loc[~meat_countries.isin(gmaps_country_series)]

meat_tofind

54             FR Yugoslavia
56             French Guiana
65                Guadeloupe
102                 Maldives
105               Martinique
123          North Macedonia
135    Republic of the Congo
146               Seychelles
164                 Tanzania
167               The Gambia
177            United States
183           Virgin Islands
Name: Country, dtype: object

In [11]:
#search for possible alternate names for countries not found in gmpas countries
for x in meat_tofind:
        possible_match = gmaps_country_series[gmaps_country_series.str.contains(x, regex=False)]
        print(f"partial matches for country {x} in meat_tofind:\n{possible_match}\n")

partial matches for country FR Yugoslavia in meat_tofind:
Series([], dtype: object)

partial matches for country French Guiana in meat_tofind:
Series([], dtype: object)

partial matches for country Guadeloupe in meat_tofind:
Series([], dtype: object)

partial matches for country Maldives in meat_tofind:
Series([], dtype: object)

partial matches for country Martinique in meat_tofind:
Series([], dtype: object)

partial matches for country North Macedonia in meat_tofind:
Series([], dtype: object)

partial matches for country Republic of the Congo in meat_tofind:
40    Democratic Republic of the Congo
dtype: object

partial matches for country Seychelles in meat_tofind:
Series([], dtype: object)

partial matches for country Tanzania in meat_tofind:
201    United Republic of Tanzania
dtype: object

partial matches for country The Gambia in meat_tofind:
Series([], dtype: object)

partial matches for country United States in meat_tofind:
205        United States of America
209    United Stat

In [12]:
#find row index of US in full meat_dataframe
meat_clean_df.loc[meat_clean_df["Country"]=="United States"]

,Country,Meat consumption,ISO-code
177,United States,124.8,USA


In [13]:
#Use US row index to update value to match gmaps country name
meat_clean_df.loc[177, "Country"] = "United States of America"

C:\Users\ermcg\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
#check that the change worked
meat_clean_df.loc[meat_clean_df["Country"]=="United States of America"]

,Country,Meat consumption,ISO-code
177,United States of America,124.8,USA


In [15]:
#create dictionary with index: country names and values: meat 
data_dict = meat_clean_df.set_index("Country").to_dict()["Meat consumption"]

In [16]:
#The following 3 boxes of code adapted from jupyter-gmaps.readthedocs.io GeoJSON example
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

# We will need to scale the data values to lie between 0 and 1
min_meat = min(data_dict.values())
max_meat = max(data_dict.values())
meat_range = max_meat - min_meat

#Convert the meat consumption data to a color
def calculate_color(meat):
    
    # make meat a number between 0 and 1
    normalized_meat = (meat - min_meat) / meat_range

    # invert meat so that high inequality gives dark color
    inverse_meat = 1.0 - normalized_meat

    # transform meat to a matplotlib color
    mpl_color = viridis(inverse_meat)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [17]:
colors = []

#for each country in gmaps, pull the correct meat stat and assign a color using defined function. 
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    try:
        meat = data_dict[country_name]
        color = calculate_color(meat)
    except KeyError:
        # no data for that country: return default color
        color = (0, 0, 0, 0.3)
    #add color to list for gmaps layer
    colors.append(color)

In [18]:
#set up gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#set up layer with colors for meat consumption data
meat_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
#add layer to base map
fig.add_layer(meat_layer)
fig

Figure(layout=FigureLayout(height='420px'))